In [10]:
import pandas as pd
import geopandas as gpd
import gdal
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
import warnings

In [ ]:
def read_DSMs(thedir, rasters, magnif, shape=False):
    
    if not shape:
        shapes = []
        for rName in rasters:
            raster = gdal.Open(f'{thedir}/{rName}')
            shapes.append(raster.GetRasterBand(1).ReadAsArray().shape)    
        out_shape = max(shapes)
    else:
        out_shape = shape
        
    dct = {}
    for rName in rasters:
        raster = gdal.Open(f'{thedir}/{rName}')
        array = raster.GetRasterBand(1).ReadAsArray()
                        
        array = np.ma.MaskedArray(array, mask=(array==array.min()))
        minval, maxval =  array.min(), array.max()
        zoomx = magnif * out_shape[0] / array.shape[0]
        zoomy = magnif * out_shape[1] / array.shape[1]
                
        # Resampling sizes using bilinear interpolation
        array = zoom(array, (zoomx, zoomy), order=1)

        array[array < minval] = 0
        array[array > maxval] = 0
        
        dct[rName[:-4]] = (raster.GetGeoTransform(), array, minval, maxval, zoomx, zoomy)
        
    print(dct.keys())
    
    return dct, out_shape

In [ ]:
def read_ortho(rasters, magnif, out_shape):
        
    dct = {}
    for rName in rasters:
        raster = gdal.Open(f'{ortho_dir}/{rName}')
        bands = []
        for i in range(1, raster.RasterCount+1):
          # Resampling sizes of each band using bilinear interpolation
            band = raster.GetRasterBand(i).ReadAsArray()
            zoomx = magnif * out_shape[0] / band.shape[0]
            zoomy = magnif * out_shape[1] / band.shape[1]
            
            band = zoom(band, (zoomx, zoomy), order=1)
            bands.append(band)

        #Stack bands back to the original multiband array
        array = np.dstack(bands)        
        minval, maxval =  array.min(), array.max()
        
        dct[rName[:-4]] = (raster.GetGeoTransform(), array, minval, maxval, zoomx, zoomy)
        
    print(dct.keys())
    
    return dct

In [4]:
def compute_periodDiff(raster_one, raster_two, period, area, rtype='dsm', oper=False):

    if not oper:
        period_diff = raster_two[1] - raster_one[1]
        warnings.filterwarnings("ignore")
        # Set 0 to pixels outside the polygon
        maxdiff_pos = raster_two[3] - raster_one[2]
        maxdiff_neg = raster_two[2] - raster_one[3]

        period_diff[period_diff < maxdiff_neg] = 0
        period_diff[period_diff > maxdiff_pos] = 0
    
    else:
        period_diff = raster_two + raster_one

    # Save as raster band
    diff_name = f'out/{rtype}_period_{period}{area}.tif'
    driver = gdal.GetDriverByName("GTiff")

    try:
        if period_diff.shape[2]>1:
        
            diff = driver.Create(diff_name, period_diff.shape[1], period_diff.shape[0],
                                 period_diff.shape[2], gdal.GDT_Float32)
        
            for i in range(period_diff.shape[2]):

                outBand = diff.GetRasterBand(i+1)
                outBand.WriteArray(period_diff[:, :, i])
                
            # Normalise the array
            period_diff = (period_diff - np.min(period_diff)) / (np.max(period_diff) - np.min(period_diff))

    except IndexError:
            diff = driver.Create(diff_name, period_diff.shape[1], period_diff.shape[0], 
                                 1, gdal.GDT_Float32)
            outBand = diff.GetRasterBand(1)
            outBand.WriteArray(period_diff)
        
    diff.SetGeoTransform(diff.GetGeoTransform())
    diff.SetProjection(diff.GetProjection())
    
    # Store in the dict
    if rtype != 'overlap':
        diff_arrays[f'{area}'] = period_diff
    else:
        diff_arrays[f'{period}{area}'] = period_diff
    
    diff = None
    
    return diff_arrays

In [5]:
def visualise(array_diffs):
    
    keys = list(array_diffs.keys())
    areas = ['A', 'B']
    for i in range(2):
        
        area = areas[i]
        val1 = array_diffs[keys[i]].copy()
        val2 = array_diffs[keys[i+2]].copy()
        
        val1[val1==0]=np.nan
        val2[val2==0]=np.nan

        fig, ax = plt.subplots(1, 2, figsize=(20, 20))
        
        im1 = ax[0].imshow(val1, cmap='viridis')
        ax[0].set_title(f'Area {area}, period 1 (sept 2020 from feb 2020)', fontsize=18)
        im2 = ax[1].imshow(val2, cmap='viridis')
        ax[1].set_title(f'Area {area}, period 2 (feb 2021 from sept 2020)', fontsize=18)
        
        if i == 0:
            fig.suptitle('Elevation difference (m)', fontsize=20, y=0.94)
            fig.colorbar(im1, ax=ax[0], orientation='horizontal', location ='top')
            fig.colorbar(im2, ax=ax[1], orientation='horizontal', location ='top')
            
        else:
            fig.colorbar(im1, ax=ax[0], orientation='vertical', location ='right', shrink=0.4)
            fig.colorbar(im2, ax=ax[1], orientation='vertical', location ='right', shrink=0.4)
    
        plt.tight_layout()

In [12]:
def compute_volumes(thedicts, array_diffs, dtype='dsm'):
    
    volume_changes = {}
    
    for idx, item in enumerate(array_diffs.items()):
         
        if dtype == 'dsm':
            key = f'dsm1_clipped_{item[0]}'
        else:
            key = f'PC1_toRas_{item[0]}'
        
        # Rescale based on zoom and interpolation
        cell_size = (thedicts[idx][0][key][0][1] * (1/thedicts[idx][0][key][4])) * (abs(thedicts[idx][0][key][0][5]) * (1/thedicts[idx][0][key][5]))
        
        vol_change = np.sum(item[1] * cell_size)
        volume_changes[item[0]] = vol_change
    
    return  pd.DataFrame.from_dict(volume_changes, orient='index')

In [ ]:
def plot_volChange(changes):
     
    fig, ax = plt.subplots(1, 2, figsize=(20,20))
    cmap = plt.cm.get_cmap('viridis').reversed()
    changes.iloc[:-2, :].plot.bar(ax=ax[0], rot=45, alpha=0.8, cmap=cmap)
    changes.iloc[-2:, :].plot.bar(ax=ax[1], rot=45, alpha=0.8, cmap=cmap)
    ax[0].set_ylabel('Volume change', fontsize=14)

    for i in range(2):
        ax[i].set_axisbelow(True)
        ax[i].set_xlabel('Area')
        ax[i].grid(axis="x")
        ax[i].grid(axis="y")
        ax[i].locator_params(nbins=50, axis='y')

    fig.suptitle('Final volume changes ($m^{3}$)', fontsize=20, y =0.95)
    plt.show()